In [1]:
import gmesh_conv.meshing
import numpy as np
import Fields.Fields as fields
import Kernels.Kernels as kernels
import BCs.BoundaryConditions as bcs
import Solvers.Solvers as solvers


In [4]:
# Get the gmsh mesh as a mesh object
mesh = gmesh_conv.meshing.mesh_from_gmsh(filename='gmesh_conv/fine.msh', orthogonalityApproach='OCA')

Info    : Reading 'gmesh_conv/fine.msh'...
Info    : 27 entities
Info    : 4712 nodes
Info    : 26239 elements


Info    : Done reading 'gmesh_conv/fine.msh'


Exception: eid =23035A surface must be either a boundary or a face.

In [3]:
# write a variable with all the volume ids of interest.
wholeProblem = [mesh.volumes[0]]  # there is only one volume and it has id of 0

In [4]:
# make a pure zero field to describe the initial conditions or initial guess of the problem.
# we use the wholeProblem before as the volume list since it is a list of volumes that contains all the elements that the field lives on.
# wholeProblem
#   volumes in the wholeProble list from above
#     elements in this volume
#       element id for this element
zeroField = fields.ICScalarField(volumeList=wholeProblem, m=mesh, fill_value=1.0)

In [5]:
# now make a scalar field - we will call this T
# use the initial condition zeroField to set the IC of the ScalarField
C = fields.ScalarField(initial_condition=zeroField, name='C')



In [6]:
# now make boundary conditions.
bc1 = bcs.DirichletBC(boundary_list=[3,4,5], m=mesh, Gamma=1.0, field=C, value=1.0)
bc2 = bcs.DirichletBC(boundary_list=[0,1,2], m=mesh, Gamma=1.0, field=C, value=0.0)


In [7]:
# then append BCs to the field of interest that this applies to
C.assign_bcs([bc1, bc2])

In [8]:
# now add a diffusion kernel to the field
diffKernel = kernels.DiffusionKernel(Gamma=1, field=C, volumeList = wholeProblem, m=mesh)
C.assign_kernels([diffKernel])

In [9]:
# make the solver object
solver = solvers.SinglePhysicsSolver(iterations=10, field=C)

In [ ]:
# solve
solver.solve()

In [11]:
# get decomposed gradient for dumping out
gradParts = C.grad.return_grad()

# get b matrix vector
bcoeffs = solver.b_to_field()
diag = solver.diag_to_field()
# get C as a Field and add to field list
fieldList = gradParts + [C, bcoeffs, diag]

# make output mesh
mesh.export_to_vtk(filename='test.vtk', fieldlist=fieldList)